In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [8]:
os.environ["LANGCHAIN_PROJECT"]= "MIRA-convo-local-dev"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [9]:
MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')
DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
        MONGODB_ATLAS_CLUSTER_URI,
        f"{DB_NAME}.{COLLECTION_NAME}",
        OpenAIEmbeddings(),
        index_name= ATLAS_VECTOR_SEARCH_INDEX_NAME
    )

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
#retriever = vector_search.as_retriever(search_kwargs={"k":5})

In [12]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )

In [14]:
chain_history = (
    contextualize_q_prompt | ChatOpenAI(model="gpt-3.5-turbo", temperature=0) | StrOutputParser() 
)

# Run
question="I am losing interest in my girlfriend"
# chain_history.invoke({"input":question})

In [15]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    chain_history | prompt_hyde | ChatOpenAI(model="gpt-3.5-turbo", temperature=0) | StrOutputParser() 
)



In [21]:
retrieval_chain = generate_docs_for_retrieval | vector_search.as_retriever(search_kwargs={"k":6})

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """Your name is MIRA, you are an empathetic psychotherapist, parenting counselor, life-coach and mental health expert. 
You need to provide the solution to the user's query.
Following are the instructions how you need to answer the Question:
 
1. Don't look for the answers from other resources. If you don't find the answer from the context, just simply answer: 
"I don't know" or "I didn't understand this, could you please provide some more information on it".

2. For a positive question, acknowledge the positivity and then give the answer. For example: 
    (a) it is so nice to see that you are in a place in life that you want to _____.
    (b) It is great to see that you are moving to the next phase of your life.
    (c) it is wonderful to know you're ready to excel in different areas of your life.

3. If it's a negative or painful question then empathize with the person that you know and you understand their pain or what they're going through 
and then give the answer. For example: We know that it must be very difficult to deal with ____.

Context: {context}


"""

human_template = """
Answer the given question based on the context given below and keeping the instruction in mind, explain each point in a simple language.


Question: {input}
"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", human_template),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(retrieval_chain, question_answer_chain)

In [27]:
from langchain_core.messages import HumanMessage

chat_history = []

#question = "What is bipolar disorder?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])


print(ai_msg_1["answer"])

I understand that it can be challenging when you start losing interest in your girlfriend. It's important to reflect on why you might be feeling this way and communicate openly with your girlfriend about your feelings. Here are some steps you can take to address this issue:

1. **Reflect on Your Feelings**: Take some time to understand why you are losing interest. Is it due to personal issues, changes in the relationship, or other factors?

2. **Open Communication**: Have an honest conversation with your girlfriend about how you are feeling. It's important to express your thoughts and listen to her perspective as well.

3. **Reignite the Spark**: Try to engage in activities that you both enjoy together to reignite the spark in your relationship. Plan dates, try new things, and spend quality time together.

4. **Seek Professional Help**: If you find it difficult to navigate your feelings or the relationship dynamics, consider seeking the help of a therapist or counselor to work through 

In [28]:
question2 = "What are the possible ways so that I can come out of this?"
ai_msg_2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question2), ai_msg_2["answer"]])


print(ai_msg_2["answer"])

It's great that you're looking for ways to address the situation you're facing. Here are some simple steps you can take to come out of feeling disinterested in your girlfriend:

1. **Reflect on Your Feelings**: Take some time to think about why you're losing interest. Is it because of something happening in your life or changes in the relationship?

2. **Open Communication**: Talk openly with your girlfriend about how you're feeling. It's important to share your thoughts and listen to her perspective too.

3. **Reignite the Spark**: Do things together that you both enjoy to bring back the excitement. Plan fun activities, go on dates, and spend quality time together.

4. **Seek Professional Help**: If you're finding it hard to understand your feelings or the relationship, consider talking to a therapist or counselor for guidance.

5. **Evaluate the Relationship**: Think about whether the relationship matches your values and goals. Be honest with yourself about what you want in a relatio